In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip3 install torch
!pip3 install transformers


     |████████████████████████████████| 368kB 4.9MB/s 
     |████████████████████████████████| 645kB 65.1MB/s 
     |████████████████████████████████| 860kB 56.3MB/s 
     |████████████████████████████████| 1.0MB 56.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=8ad6e93c6a07ef321259d64da3c11eaeaae5a2e6b343246c07cb76113c00af77
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [4]:
import torch
from transformers import BertTokenizer, BertModel, BertForTokenClassification, BertConfig
from keras.preprocessing.sequence import pad_sequences
import sys
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import pandas as pd
import numpy as np
from itertools import combinations_with_replacement, permutations
import datetime
from torch.nn.functional import softmax
import itertools

Using TensorFlow backend.


In [5]:
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)
cuda0 = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(cuda0)

cuda


In [6]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=8e825ce1a4a4e251257f2295786a6781dcdfa071ec48e83cb8cbdbb74583d421
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [7]:
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.6 GB  | Proc size: 502.5 MB
GPU RAM Free: 16270MB | Used: 10MB | Util   0% | Total 16280MB


In [0]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from inspect import signature
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc

def plot_AUCPRC(labels, predictions):
	"""labels: a 1d list of all the labels (0 or 1) for our samples
		predictions: a 1d list of all the probabilities (between 0 and 1)
		             for each one of our samples. It should be the probability that
		             that this sample belongs to the second class (e.g a splice site.)
	"""
	# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument\
	precision, recall, _ = precision_recall_curve(labels, predictions)
	auc = auc(recall, precision)
	step_kwargs = ({'step': 'post'}
	               if 'step' in signature(plt.fill_between).parameters
	               else {})
	plt.step(recall, precision, color='b', alpha=0.2,
	         where='post')
	plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

	plt.xlabel('Recall')
	plt.ylabel('Precision')
	plt.ylim([0.0, 1.05])
	plt.xlim([0.0, 1.0])
	plt.title('2-class Precision-Recall curve: auc={0:0.2f}'.format(
	          auc))

In [0]:
def process_data(path, kmer_type):
    first = True
    file_table = []
    with open(path) as fp:
        i = 0
        for line in fp:
            if not first:
                new_entry = []
                split_line = line.split()
                new_entry.append(split_line[0])
                new_entry.append(split_line[1])
                sep = ' '
                kmers = list(map(lambda x: x.replace(']', '').replace('[', '').replace(',','').replace('\'', ''), split_line[2:1002]))
                if '--PAD--' in kmers:
                    kmers = kmers[:kmers.index('--PAD--')]
                kmers_string = sep.join(kmers)
                new_entry.append(kmers_string)
                new_entry.append(list(map(lambda x: int(x.replace(']', '').replace('[', '').replace(',','').replace('.0','').replace('\'','')), split_line[1002:])))
                file_table.append(new_entry)
                i += 1
            first = False
    
    return handle_tokenization(file_table, kmer_type)

def handle_tokenization(file_table, kmer_type):
    MAX_LEN = 1002
    fourmers_path = '/content/drive/My Drive/k_mers/fourmers.txt'
    hexamers_path = '/content/drive/My Drive/k_mers/hexamers.txt'
    octamers_path = '/content/drive/My Drive/k_mers/octamers.txt'
    kmers_path = fourmers_path
    if kmer_type == 'hexamers':
      kmers_path = hexamers_path
    elif kmer_type == 'octamers':
      kmers_path = octamers_path
    formatted_hexamers = ['[CLS] ' + f[2] + ' [SEP]' for f in file_table]
    labels = [[0] + f[3] + [0] for f in file_table]
    tokenizer = BertTokenizer(kmers_path, max_len=MAX_LEN)
    attention_masks = [np.concatenate([np.ones((len(a.split()))), np.zeros((MAX_LEN - len(a.split())))]) for a in formatted_hexamers]
    
    return tokenizer, formatted_hexamers, attention_masks, labels



def gradient_descent(l):
    optimizer.zero_grad()
    l.backward()
    optimizer.step()

def write_data(batch, batch_size, labels_array, total_loss, state_path, save_state=False):
    if batch > 0 and batch % (500 // batch_size) == 0:
      
      last_seq_path = "/content/drive/My Drive/bert_last_i.txt"
      with open(last_seq_path, 'w+') as seq_record:
        seq_record.write(str(batch))
      loss_trace_path = "/content/drive/My Drive/bert_loss_trace.txt"
      with open(loss_trace_path, 'a+') as loss_record:
        loss_record.write(str(batch) + ' loss: ' + str(total_loss/100) + '\n')
      if save_state:
        torch.save(model.state_dict(), state_path)
    if batch > 0 and batch % (100 // batch_size) == 0:
      print("Batch: " + str(batch) + " loss: " + str(total_loss / 100))
      total_loss = 0
    return total_loss

In [0]:
batch_size = 1
MAX_LEN = 1002
FOURMERS = 'fourmers'
HEXAMERS = 'hexamers'
OCTAMERS = 'octamers'

tokenizer, formatted_hexamers, attention_masks, labels = process_data('/content/drive/My Drive/partitioned_samples/split_samples_8-mer/split_samples_8-mer_train.txt', OCTAMERS)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0) 

model = BertForTokenClassification(BertConfig.from_json_file('/content/drive/My Drive/bert_configuration.json'))
model.resize_token_embeddings(len(tokenizer))
model.to(device)

#model.load_state_dict(torch.load("/content/drive/My Drive/bert_splice_weights_2.pt"))
last_i = 0
with open('/content/drive/My Drive/bert_last_i.txt', 'r') as last_i_file:
  i = last_i_file.read()
  last_i = int(i)

print(last_i)
optimizer = Adam(model.parameters(), lr=3e-5)

training_loss = []
model.train()
class_weights = torch.tensor(np.array([1.0, 165.0])).float().cuda()
loss = CrossEntropyLoss(weight=class_weights)
print("ready to train")
np.random.shuffle(formatted_hexamers)
total_loss = 0
predictions_array = []
labels_array = []
for batch in range(len(formatted_hexamers) // batch_size):
    seq_batch = formatted_hexamers[batch*batch_size:(batch+1)*batch_size]
    attention_masks_batch = attention_masks[batch*batch_size:(batch+1)*batch_size]
    attention_masks_batch = torch.tensor(attention_masks_batch).long().cuda()
    split_seq_batch = seq_batch[0].split()
    batch_ids = [tokenizer.convert_tokens_to_ids(split_seq_batch)]
    batch_ids = pad_sequences(batch_ids, maxlen=MAX_LEN, truncating='post', padding='post')[0]
    batch_ids = torch.tensor(batch_ids).unsqueeze(0).long().cuda()
    batch_labs = labels[batch*batch_size:(batch+1)*batch_size]
    flat_labs = list(itertools.chain(*batch_labs))
    labels_array.extend(flat_labs)
    #print(np.shape(labels_array))
    batch_labels = torch.tensor(batch_labs).long().cuda()

    
    predictions = model(batch_ids, attention_mask=attention_masks_batch)
    softmax_preds = softmax(predictions[0], dim=2).cpu().detach().numpy()
    flat_softmax = np.array(list(itertools.chain(*softmax_preds)))
    predictions_array.extend(list(flat_softmax[:,-1:]))
    #print(np.shape(predictions_array))
    l = loss(predictions[0].squeeze(0), batch_labels[0])
    total_loss += l.item()
    training_loss.append(l.item())
    
    gradient_descent(l)
    state_path = "/content/drive/My Drive/bert_splice_weights/octamers_model_weights.pt"
    total_loss = write_data(batch, batch_size, labels_array, total_loss, state_path, save_state = True)

#plot_AUCPRC(labels_array, predictions_array)
    


3000
ready to train
Batch: 100 loss: 0.8090678329579533
Batch: 200 loss: 0.7973055811226368
Batch: 300 loss: 0.6902930942177773
